In [1]:
# -- coding: utf-8 --
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
sess = tf.InteractiveSession()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
def weight_variable(shape):
    #初始化权重W参数，给权重制造一些随机噪声打破完全对称
    #此处生成的是包含噪声的正态分布，噪声标准差设置为0.1
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)
    
def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

In [3]:
def conv2d(x,W):
    return tf.nn.conv2d(x,W, strides = [1,1,1,1],padding = 'SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize = [1,2,2,1],strides = [1,2,2,1],
                          padding = 'SAME')

In [4]:
#准备输入数据
x = tf.placeholder(tf.float32,[None,784])
y_ = tf.placeholder(tf.float32,[None,10])
#卷积神经网络要用到空间结构信息，将1维输入向量转为2维的图片结构
#  1x784 -> 28x28  
#  [-1,28,28,1] 其中-1代表样本数量不固定，最后1代表颜色通道数量，若为彩色则为3
x_image = tf.reshape(x,[-1,28,28,1])

In [5]:
#开始定义第一个卷积层。
#[5,5,1,32]卷积核尺寸5x5,1个颜色通道，32个不同的卷积核
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [6]:
#第二个卷积层，卷积核数量为64，即这一层的卷积会提取64种特征
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [7]:
#经过前面2次步长2x2的最大池化，变长为原来的1/4，即7x7
#第二个卷积层的卷积核数量为64，输出的tensor尺寸为7x7x64
#因输出为1维向量，故将其reshape为1维向量
W_fc1 = weight_variable([7*7*64,1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2,[-1, 7*7*64])
#连接一个全连接层，隐藏节点为1024，使用relu激活函数
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

In [8]:
#减轻过拟合，使用一个dropout层
#Dropout用法：通过一个placeholder传入keep_prob比率来控制
#训练时随机丢弃一部分节点减轻过拟合
#但预测时则保留全部数据来追求最好的预测性能
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [9]:
#最后将dropout层连接到softmax层得到最后概率并输出
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + b_fc2)

In [10]:
#定义LOSS FUNCTION为cross_entropy  优化器使用Adam,学习率 1e-4
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ *tf.log(y_conv),
                                             reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [11]:
#定义评测准确率的操作
correct_prediction = tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [13]:
#开始训练   依次初始化所有参数   keep_prob=0.5   
# 使用大小50的mini-batch 进行共20000次训练迭代，参与训练样本数量共100W
# 每100次训练 对准确率进行一次评测（评测时keep_prob设置为1）
tf.global_variables_initializer().run()
for i in range(2000):#20000
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict = {x:batch[0],y_:batch[1],
                                                   keep_prob:1.0})
        print("step %d,training accuracy is %g"%(i,train_accuracy))
    train_step.run(feed_dict = {x:batch[0],y_:batch[1],keep_prob:0.5})

step 0,training accuracy is 0.14
step 100,training accuracy is 0.82
step 200,training accuracy is 0.92
step 300,training accuracy is 0.9
step 400,training accuracy is 0.96
step 500,training accuracy is 0.96
step 600,training accuracy is 0.92
step 700,training accuracy is 0.98
step 800,training accuracy is 0.96
step 900,training accuracy is 1
step 1000,training accuracy is 0.96
step 1100,training accuracy is 0.98
step 1200,training accuracy is 0.96
step 1300,training accuracy is 1
step 1400,training accuracy is 1
step 1500,training accuracy is 0.98
step 1600,training accuracy is 0.98
step 1700,training accuracy is 1
step 1800,training accuracy is 1
step 1900,training accuracy is 0.98


In [ ]:
#全部训练完成后，在最终的测试集上进行全面测试，得到整体的分类准确率
print("test accuracy %g"%accuracy.eval(feed_dict={x:mnist.test.images,
                                                 y_:mnist.test.labels,
                                                 keep_prob:1.0}))